| | |
|--|--|
| **Names** | *Andrea Papenmeier, Waheed ud din Siddiqui* |
| **Group** | *ML_HMI 02* |

In [1]:
import numpy as np
import operator

# Decision Trees

In this lab, we will implement the C4.5 algorithm for classification with decision trees. 


## Data

Let's start by loading the datasets that we will be working with. The ```mushroom``` dataset is a dataset describing the features of different species of mushrooms, and the goal is to derive simple rules to predict whether a species is poisonous or not.

In [2]:
# This code is given
data = np.load("uci-mushrooms.npz")
print data.keys()

d = data['inputs']
l = data['targets']
fname = data['feature_names']
print(l.size)
#check = []
#for i in range(fname.shape[0]):
#    check.append(np.sum(d[:,i] == '?'))
#print(check)
#print(d[:,10])

['inputs', 'targets', 'feature_names']
8416


## Helper functions

Here are a few functions to deal with categorical labels

In [3]:
# This code is given

def listValues(inputlist):
    """List the possible values of a given feature in a dataset"""
    values = set([])
    for x in inputlist:
        values.add(x)
    return list(values)

def countLabels(labelList):
    """Return the number of labels in a list"""
    counts = {}
    for x in labelList:
        if x not in counts:
            counts[x] = 1
        else:
            counts[x] += 1
    return counts

def majorityLabel(labelList):
    counts = countLabels(labelList)
#    print "Majority label:", counts
    return max(counts.iteritems(), key=operator.itemgetter(1))[0]

for value in listValues(l):
    targetIndicesTrue  = l == value
    print(np.sum(targetIndicesTrue))

3928
4488


In [4]:
# This assignment was somewhat unclear, we understood it such that we should calculate the entropy of a dataset
# given the class labels (slide 12 of lecture slides week 6)

def entropy(labelList):
    
    # count all elements
    nbrTotalElements = len(labelList)
    
    # count instances in class 1
    nbrClass1 = countLabels(labelList)[(listValues(labelList)[0])]
    # count instances in class 2
    nbrClass2 = nbrTotalElements-nbrClass1
    if ((nbrClass1 == nbrTotalElements) or (nbrClass2 == nbrTotalElements)):
        res = 0
    else:
        res = 0-(((nbrClass1+0.0)/nbrTotalElements)*np.log2(((nbrClass1+0.0)/nbrTotalElements)))-(((nbrClass2+0.0)/nbrTotalElements)*np.log2(((nbrClass2+0.0)/nbrTotalElements)))
    
    return res

print entropy(l)
               

0.996803828522


## Computing the entropy of a list of categorical values

The decision tree makes its decision based on decision nodes, and these nodes are evaluated based on information gain they provide on the training data. Information gain is computed as 
$$
IG = H(parent) - \sum_j \frac{N_j}{N} H(child=j)
$$
the entropy of the data associated with the parent, minus the weighted entropy of the data-subsets associated with the chidlren

**question 1 [5 credits]]** Implement a helper function that computes the entropy of a set of categorical labels


## Implementing the tree

Below, you are given the code to make a decision tree, train it and classify datapoints using binary decision nodes based on categorical variables. The implementation includes  ```Leaf``` nodes, wich return a classification result, and ```BinaryNode```, which decide which child should be responsible for the further classification.

The ```BinaryNode``` constructor selects the most informative value in the data it's trained on, and makes a decision based on whether or not the feature of the datapoint to be classified equals that value, or not.

In [6]:
# This code is given

class Node:
    def classify(self, datapoint):
        """Return the classification label for the given datapoint"""
        raise Exception("Not Implemented")        
    def entropy(self):
        """Return the entropy of the training data associated with this node"""
    def informationGain(self):
        """Return the information gain of this node on the training data """
        raise Exception("Not Implemented by derived class")

class Leaf(Node):
    def __init__(self,inputs, targets):
        self.v = majorityLabel(targets)
        self.inputs = inputs
        self.targets = targets
        self.children = None
        self.H = entropy(targets)
        
    def entropy(self):
        return self.H
    
    def informationGain(self):
        return 0.
    
    def classify(self,datapoint):
        return self.v
    
    def __str__(self):
        return "\t\t\t=> %s" % (self.v)
    
class BinaryNode(Node):
    """Make a decision based on whether a given feature equals a given value, or not"""
    def __init__(self, inputs, targets, feature):
        self.feature = feature
        self.value   = None
        self.IG      = 0.0
        self.H       = entropy(targets)                     # Entropy of unsplit data
        self.inputs  = inputs
        self.targets = targets
        N = float(targets.size)                             # Total number of datapoints

        for value in listValues(inputs[:,feature]):
            indicesTrue  = inputs[:,feature]==value
            indicesFalse = inputs[:,feature]!=value

            if indicesTrue.all() or indicesFalse.all():
                continue

            children = [ Leaf(inputs[indicesTrue,:], targets[indicesTrue]),
                         Leaf(inputs[indicesFalse,:], targets[indicesFalse]) ]
            conditions = [ "%s==%s" % (fname[feature], value), "%s!=%s" % (fname[feature], value) ]

            Nt = float(children[0].targets.size)        # N.o. datapoints in True condition
            Nf = float(children[1].targets.size)        # N.o. datapoints in False condition
            pt = Nt/N
            pf = Nf/N
            ig = self.H - pt * children[0].entropy() - pf * children[1].entropy()
            
            if ig > self.IG:
                self.value = value
                self.IG = ig
                self.children = children
                self.conditions = conditions
    
    def entropy(self):
        return self.H
    
    def informationGain(self):
        """Return the entropy gain of splitting the given data according to 
        the specified feature and value"""
        return self.IG
    
    def classify(self, x):
        if x[self.feature] == self.value:
            return self.children[0].classify(x)
        else:
            return self.children[1].classify(x)
        
    def __str__(self):
        return " => Binary decision: '%s =?= %s'" % (fname[self.feature], self.value )
    
    

In [7]:
# This code is given

def c45(inputs, targets, minSize=100):
    ig = 0.
    res = None
    for feature in range(inputs.shape[1]):
        n = BinaryNode(inputs,targets,feature)
        if n.informationGain() > ig:
            ig = n.informationGain()
            res = n
    if not res:
        print "WARNING: No informative feature found (Fishy!)"
        return res
    for i in range(len(res.children)):
        if res.children[i].entropy() > 0. and res.children[i].targets.size>minSize:
            c = c45(res.children[i].inputs, res.children[i].targets, minSize)
            if c:
                res.children[i]=c
    return res

root = c45(d,l)

In [12]:
# This code is given

def prettyPrint(tree, cond="", dlist = [True]):
    indent = ""
    for d in dlist[:-1]:
        indent += "   " if d else "|  "
    print indent + "+-> " + cond + " " + tree.__str__()
    if tree.children:
        for cond,cld in zip(tree.conditions[:-1],tree.children[:-1]):
            prettyPrint(cld,cond,dlist + [ False ])
        prettyPrint(tree.children[-1], tree.conditions[-1], dlist+[ True ])

    

prettyPrint(root)


+->  => n-ary decision: stalk-shape = ?
   +-> stalk-shape==ENLARGING => n-ary decision: gill-size = ?
   |  +-> gill-size==BROAD => n-ary decision: gill-attachment = ?
   |  |  +-> gill-attachment==ATTACHED 			=> EDIBLE
   |  |  +-> gill-attachment!=FREE => n-ary decision: gill-spacing = ?
   |  |     +-> gill-spacing==CLOSE => n-ary decision: Number-of-rings = ?
   |  |     |  +-> Number-of-rings==NONE 			=> POISONOUS
   |  |     |  +-> Number-of-rings!=NONE 			=> EDIBLE
   |  |     |  +-> Number-of-rings!=ONE => n-ary decision: cap-shape = ?
   |  |     |     +-> cap-shape==FLAT => n-ary decision: habitat = ?
   |  |     |     |  +-> habitat==PATHS 			=> POISONOUS
   |  |     |     |  +-> habitat!=PATHS 			=> POISONOUS
   |  |     |     |  +-> habitat!=GRASSES 			=> POISONOUS
   |  |     |     +-> cap-shape!=FLAT => n-ary decision: habitat = ?
   |  |     |     |  +-> habitat==GRASSES 			=> POISONOUS
   |  |     |     |  +-> habitat!=GRASSES 			=> POISONOUS
   |  |     |     |  +-> 

# Training 

The code above trains a decision tree.

**Question 2 [5 credits]** Split the ```mushroom``` dataset into 10 folds and perform 10-fold cross-validated training and testing of the decision tree. Report de tree's performance in terms of average precision, recall and accuracy.

**Question 3 [5 credits]** Modify the training function provided above to stop splitting nodes when the corresponding training data contains less than ```minSize``` datapoints, where ```minSize``` is a parameter to the function. Split the data into 80\% train and 20\% test, and use 10-fold cross-validation on the training set to find a good value for this parameter. Then report the confusion matrix, precision, recall and accuracy that you obtain on the test set.


In [8]:
# Answer to Q2
from sklearn.cross_validation import KFold

def kfoldsvalidation(datapoints, labels, foldNbr, minSizeNodes):
    """function to perform k-fold cross-validation with varying minSize in c45 function"""
    #help(KFold)
    # divide dataset into 10 times training+test data
    kf = KFold(len(labels), n_folds=foldNbr)
    X = datapoints
    y = labels

    # setup array to store subsets and trees
    subsets = []
    root = []

    # loop through all 10 cross validation sets and store them in array
    for train_index, test_index in kf:
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        subsets.append(((X_train, y_train), (X_test, y_test)))

    #print len(subsets) # == 10 --> we have 10 subsets
    #print len(subsets[0]) # == 2 --> we each subset has a subsubset of train data and one of test data
    #print len(subsets[0][0]) # == 2 --> each subsubset has a set of labels and a set of 22 feature values
    
    # construct trees and store in root array
    i = 0
    while i<len(subsets):
        dat = subsets[i][0][0]
        lab = subsets[i][0][1]
        root.append(c45(dat, lab, minSizeNodes))
        i += 1
    
    # output a set of trees and subsets
    return root, subsets

roots = kfoldsvalidation(d, l, 10, 100)



In [9]:
# Continuation of answer to Q2
# Continuation of answer to Q2

# get numbers for correct and wrong predictions per class
def confusionMatrix(labels, predictions, class1): 
    Act1Pred1 = 0
    Act1Pred2 = 0
    Act2Pred2 = 0
    Act2Pred1 = 0
    i = 0
    while i<len(labels):
        if labels[i] == class1:
            if predictions[i] == class1:
                Act1Pred1 += 1            # class 1 elements that have been predicted correctly
            else:
                Act1Pred2 += 1            # class 1 elements that have been predicted incorrectly
        else:
            if predictions[i] == class1:
                Act2Pred1 += 1            # class 2 elements that have been predicted incorrectly
            else:
                Act2Pred2 += 1            # class 2 elements that have been predicted correctly
        i += 1
    return Act1Pred1, Act1Pred2, Act2Pred2, Act2Pred1

def drawMatrix(matrixValues):
    
    print '\t' + '\t' + 'Pred.Class1' + '\t' + 'Pred.Class2'
    print 'Act.Class1' + '\t' + '\t' + repr(matrixValues[0]) + '\t' + '\t' + repr(matrixValues[1])
    print 'Act.Class2' + '\t' + '\t' + repr(matrixValues[3]) + '\t' + '\t' + repr(matrixValues[2])
    
    #t.add_rows([['', 'Pred.: Class1', 'Pred.: Class2'], ['Act.: Class1', repr(matrixValues[0]), repr(matrixValues[1])], ['Act.: Class2', repr(matrixValues[3]), repr(matrixValues[2])]])
    #print t.draw()

# get precision: ratio of actual class c elements to predicted class c elements
# '0.0+' is added to ensure it is stored as a float, not an integer, to avoid rounding errors
def precision(ActCPredC, ActElsePredC):
    return (0.0+ActCPredC)/(ActElsePredC+ActCPredC)

# get recall: 
def recall(ActCPredC, ActCPredElse):
    return (0.0+ActCPredC)/(ActCPredElse+ActCPredC)
    
# get accuracy: 
def accuracy(ActCPredC, ActElsePredElse, ActCPredElse, ActElsePredC):
    return (0.0+ActCPredC+ActElsePredElse)/(ActCPredC+ActElsePredElse+ActCPredElse+ActElsePredC)


def getStatistics(trees, subs, initialLabelList):
    """takes constructed trees, subsets, and an initial label list as input
    subs: subsets with the structure: 
    subs[i] -> i=number of k-folds, subs[i][j] -> j=2 training+test set, subs[i][j][k] -> k=2 data+labels, subs[i][j][k][l] -> l=actual data points"""
    
    class1 = listValues(initialLabelList)[0]            # define class1 value as first value of label list of initial data set
    # averages class 1
    averagePrec1 = 0.0
    averageRec1 = 0.0
    # averages class 2
    averagePrec2 = 0.0
    averageRec2 = 0.0
    # average accuracy (accuracy is not w.r.t. a specific class)
    averageAcc = 0.0
    
    # from here on everything concerns ONE specific subset
    a = 0
    while a<len(subs):
        da = subs[a][1][0]
        la = subs[a][1][1]
        testSetSize = len(da)
        prediction = []
    
    
        # predict class labels in test set for each test data point
        i = 0
        while i<testSetSize:
            prediction.append(trees[a].classify(da[i]))
            i += 1
            
        # get confusion matrix
        matrix = confusionMatrix(la, prediction, class1)
        #print matrix
        
        # get precision for class 1
        prec = precision(matrix[0], matrix[3])
        averagePrec1 += prec
        #print prec
        # get precision for class 2
        prec = precision(matrix[2], matrix[1])
        averagePrec2 += prec
        #print prec
        
        # get recall for class 1
        rec = recall(matrix[0], matrix[1])
        averageRec1 += rec
        #print rec
        # get recall for class 2
        rec = recall(matrix[2], matrix[3])
        averageRec2 += rec
        #print rec
        
        # get accuracy
        acc = accuracy(matrix[0], matrix[1], matrix[2], matrix[3])
        averageAcc += acc
        #print acc
        
        a+=1
        
    averagePrec1 /= len(subsets) 
    averageRec1 /= len(subsets)
    averagePrec2 /= len(subsets) 
    averageRec2 /= len(subsets)

    averageAcc /= len(subsets)

    # average precisions for class 1 and for class 2
    print 'precision class1: ' + repr(averagePrec1) 
    print 'precision class2: ' + repr(averagePrec2)
    # average recalls for class 1 and for class 2
    print 'recall class1: ' + repr(averageRec1)
    print 'recall class2: ' + repr(averageRec2)
    # average accuracy
    print 'accuracy: ' + repr(averageAcc)
    
getStatistics(roots[0], roots[1], l)

averages of all 10 folds together:
precision:
0.995899238103 1.0
recall:
1.0 0.996468327916
accuracy:
0.466730167965


In [ ]:
# Answer to Q3



# Different types of decisions

Our tree for now can only make decisions based on binary conditions, but our dataset contains features with more than two possible values. 

**Question 4 [10 credits]** Implement a new type of node, ```MultiNode``` that has a child per possible value of the feature. Be careful to deal with the possibility that the test data may contain categories that the training set didn't have.

In [13]:
# This code must be modified

class MultiNode(Node):
    """Make a decision based the value of the given feature, i.e. as many children as there are 
    possible values for the feature"""
    def __init__(self, inputs, targets, feature):
        # YOUR CODE COMES HERE
        self.feature = feature
        self.value   = []                                   #each child would have its own value
        self.IG      = 0.0
        self.H       = entropy(targets)                     # Entropy of unsplit data
        self.inputs  = inputs
        self.targets = targets
        N = float(targets.size)                             # Total number of datapoints

        childList = []                                      #more than 2 children now
        conditionList = []                                  #each childre has a condition
        weightedEntropy = 0.0
        
        for value in listValues(inputs[:,feature]):
            trueIndices = inputs[:,feature]==value
            if trueIndices.all():
                continue
            childList += [Leaf(inputs[trueIndices,:], targets[trueIndices])]
            conditionList += ["%s==%s" % (fname[feature], value), "%s!=%s" % (fname[feature], value)]
            self.value += [value]
            Nt = float(childList[-1].targets.size)  #number of elements in trueIndices
            Pt = Nt/N                              #probability of truth due to this child
            weightedEntropy -= Pt *  childList[-1].entropy()
        ig = self.H - weightedEntropy
        self.IG = ig
        self.children = childList
        self.conditions = conditionList
       
        
    def entropy(self):
        return self.H
    
    def informationGain(self):
        """Return the entropy gain of splitting the given data according to 
        the specified feature and value"""
        return self.IG
    
    def classify(self, x):
        # YOUR CODE COMES HERE
        for i in range(0, len(self.value)):
            if x[self.feature] == self.value:
                return self.childList[i].classify(x)
        return self.value       #this is what it will return for unseen data while testing
        
    def __str__(self):
        return "=> n-ary decision: %s = ?" % (fname[self.feature])

## Dealing with heterogeneous features

The code below keeps a list of the type of node that is applicable for each feature, in this case ```BinaryNode``` or ```Multinode```. 

**Question 5 [5 credits]** Is a tree of multinodes better than a tree of binary nodes? Is it more compact? Is it generalising better?

*Answer to Q5:*
It is better because even though it increases the width (breadth/ spread of branches), The entropies are calculated at every step and it reduces the overall error of the leaf nodes. It may or may not reduce the complexity of the tree though. In this case it does not reduce the tree complexity.

In [14]:
#This code is given but minSize is not implemented

featureType = [ MultiNode for i in range(l.size) ]
def c45(inputs, targets, minSize, featureType):
    ig = 0.
    res = None
    for feature in range(inputs.shape[1]):
        n = featureType[feature](inputs,targets,feature)
        if n.informationGain() > ig:
            ig = n.informationGain()
            res = n
    for i in range(len(res.children)):
        if res.children[i].entropy() > 0. and res.children[i].targets.size>minSize:
            res.children[i] = c45(res.children[i].inputs, res.children[i].targets, minSize, featureType)
    return res

root = c45(d,l,500, featureType)
prettyPrint(root)


##########
#def c45(inputs, targets, minSize=100):
#    ig = 0.
#    res = None
#    for feature in range(inputs.shape[1]):
#        n = BinaryNode(inputs,targets,feature)
#        if n.informationGain() > ig:
#            ig = n.informationGain()
#            res = n
#    if not res:
#        print "WARNING: No informative feature found (Fishy!)"
#        return res
#    print(len(res.children))
#    for i in range(len(res.children)):
#        if res.children[i].entropy() > 0. and res.children[i].targets.size>minSize:
#            c = c45(res.children[i].inputs, res.children[i].targets, minSize)
#            if c:
#                res.children[i]=c
#    return res

#root = c45(d,l)

+->  => n-ary decision: stalk-shape = ?
   +-> stalk-shape==ENLARGING => n-ary decision: gill-size = ?
   |  +-> gill-size==BROAD => n-ary decision: gill-attachment = ?
   |  |  +-> gill-attachment==ATTACHED 			=> EDIBLE
   |  |  +-> gill-attachment!=FREE => n-ary decision: gill-spacing = ?
   |  |     +-> gill-spacing==CLOSE => n-ary decision: Number-of-rings = ?
   |  |     |  +-> Number-of-rings==NONE 			=> POISONOUS
   |  |     |  +-> Number-of-rings!=NONE 			=> EDIBLE
   |  |     |  +-> Number-of-rings!=ONE => n-ary decision: cap-shape = ?
   |  |     |     +-> cap-shape==FLAT => n-ary decision: habitat = ?
   |  |     |     |  +-> habitat==PATHS 			=> POISONOUS
   |  |     |     |  +-> habitat!=PATHS 			=> POISONOUS
   |  |     |     |  +-> habitat!=GRASSES 			=> POISONOUS
   |  |     |     +-> cap-shape!=FLAT => n-ary decision: habitat = ?
   |  |     |     |  +-> habitat==GRASSES 			=> POISONOUS
   |  |     |     |  +-> habitat!=GRASSES 			=> POISONOUS
   |  |     |     |  +-> 

In [ ]:
# Playground for answering Q5
######Question already answered

## Continuous values

Until now, we have only dealt with categorical features. In the next part of the lab, you will deal with mixed categorical and continuous features.

One way to deal with continuous features is to split the dataset based on whether the feature is larger than a certain threshold, or not. The number of possible threshold values to consider is, of course, infinite in theory, but when deciding what the threshold value should be, based on a training dataset, only threshold values that will result in a different classification of those training examples matter: those are values that are smaller than one datapoint's feature value and larger than another's. Moreover, when considering a threshold in between two feature values, any value of the threshold in that range will result in the same classification, but to maximise generalisation it makes sense to maximise the margin: that is, to take a threshold value that is in the middle between the two.

Also note that it only makes sense to consider ranges of feature values that are disjoint, i.e., consider thresholds that are in the middle between neighbouring datapoints (along that feature's dimension), not in the middle between any pair of datapoints.

**Question 6 [10 credits]**. You guessed it, implement a node ```ContNode``` that selects a threshold value based on its training subset and classifies new datapoints based on whether they are smaller than that value, or not.

In [15]:
# This code must be modified.

class ContNode(Node):
    """Make a binary decision based on a threshold, for the value of the given feature"""
    def __init__(self, inputs, targets, feature):
         # YOUR CODE COMES HERE
        self.feature = feature
        self.value   = None
        self.IG      = 0.0
        self.H       = entropy(targets)                     # Entropy of unsplit data
        self.inputs  = inputs
        self.targets = targets
        N = float(targets.size)                             # Total number of datapoints
        threshold = 0.0
        i = 0
        for value in listValues(targets):
            targetIndicesTrue  = targets == value
            threshold += np.mean(inputs[targetIndicesTrue,feature])    
            i += 1
        threshold = threshold/i
        indicesTrue  = inputs[:,feature] >= threshold
        indicesFalse = inputs[:,feature] < threshold

        if indicesTrue.all() or indicesFalse.all():
            threshold = 0.0
            ig =0.0
        else:    
            children = [ Leaf(inputs[indicesTrue,:], targets[indicesTrue]),
                        Leaf(inputs[indicesFalse,:], targets[indicesFalse]) ]
            conditions = [ "%s>=%s" % (fname[feature], value), "%s<%s" % (fname[feature], value) ]

            Nt = float(children[0].targets.size)        # N.o. datapoints in True condition
            Nf = float(children[1].targets.size)        # N.o. datapoints in False condition
            pt = Nt/N
            pf = Nf/N
            ig = self.H - pt * children[0].entropy() - pf * children[1].entropy()
        if ig > self.IG:
            self.value = threshold
            self.IG = ig
            self.children = children
            self.conditions = conditions
    
                
    def entropy(self):
        return self.H
    
    def informationGain(self):
        """Return the entropy gain of splitting the given data according to 
        the specified feature and value"""
        return self.IG
    
    def classify(self, x):
        if x[self.feature] >= self.value:
            return self.children[0].classify(x)
        else:
            return self.children[1].classify(x)
        
    def __str__(self):
        return "=> threshold decision: %s >= %f" % (fname[self.feature],self.value)
    

## Classification based on mixed features

The following dataset contains 14 features describing people's situation, and the classification task is to predict whether they earn more than \$50k a year or not. The categorical decision nodes we implemented above can work with any representation of categorical data, but the continuous decision node requires a numerical representation of the feature. Numpy arrays cannot easily contain both strings and numbers, and so to simplify our life, we converted the dataset to 

To allow our classifier to work with mixed features, we use a numerical representation of the categorical features in the following. So, let's first load the data:

In [16]:
# This code is given

income = np.load('income.npz')
print income.keys()
inputs = income['inputs']
targets = income['targets']
fname = income['fnames']
ftypes = income['ftypes']
print(np.size(inputs))

['inputs', 'catindices', 'fnames', 'targets', 'ftypes']
455854


**Question 7 [10 credits]** Take 1000 training datapoints at random from the dataset, and another 1000 test points that are distinct from those (don't mix train and test). Train a decision tree on the train set and report the confusion matrix obtained on the test set. What are the obtained precision, recall and accuracy? 

In [32]:
# This code is given
import random
def confusionMatrix(labels, predictions, class1): 
    Act1Pred1 = 0
    Act1Pred2 = 0
    Act2Pred2 = 0
    Act2Pred1 = 0
    i = 0
    while i<testSetSize:
        if labels[i] == class1:
            if predictions[i] == class1:
                Act1Pred1 += 1            # class 1 elements that have been predicted correctly
            else:
                Act1Pred2 += 1            # class 1 elements that have been predicted incorrectly
        else:
            if predictions[i] == class1:
                Act2Pred1 += 1            # class 2 elements that have been predicted incorrectly
            else:
                Act2Pred2 += 1            # class 2 elements that have been predicted correctly
        i += 1
    return Act1Pred1, Act1Pred2, Act2Pred2, Act2Pred1

# get precision: ratio of actual class c elements to predicted class c elements
# '0.0+' is added to ensure it is stored as a float, not an integer, to avoid rounding errors
def precision(ActCPredC, ActElsePredC):
    return (0.0+ActCPredC)/(ActElsePredC+ActCPredC)

# get recall: 
def recall(ActCPredC, ActCPredElse):
    return (0.0+ActCPredC)/(ActCPredElse+ActCPredC)
    
# get accuracy: 
def accuracy(ActCPredC, ActElsePredElse, ActCPredElse, ActElsePredC):
    return (0.0+ActCPredC+ActElsePredElse)/(ActCPredC+ActElsePredElse+ActCPredElse+ActElsePredC)

featureType = [ ContNode if ftypes[i]=='continuous' else BinaryNode for i in range(len(ftypes)) ]
#tree = c45(inputs[:10000,:],targets[:10000],100,featureType)
testDataPoints = random.sample(range(10000), 1000)
trainigDataPoints = (random.sample(range(20000,30000), 1000))
testDataInputs = inputs[testDataPoints,:]
testDataTargets = targets[testDataPoints]
trainingDataInputs = inputs[trainigDataPoints,:]
trainingDataTargets = targets[trainigDataPoints]

#prettyPrint(tree)
tree = c45(trainingDataInputs,trainingDataTargets,100,featureType)
class1 = listValues(testDataTargets)[0]
prediction = []
i = 0
while i< (1000):
    prediction.append(tree.classify(testDataInputs[i]))
    i += 1
myMatrix = confusionMatrix(testDataTargets, prediction, class1)
precisionClass1 = precision(myMatrix[0], myMatrix[3])
precisionClass2 = precision(myMatrix[2], myMatrix[1])
recallClass1 = recall(myMatrix[0], myMatrix[1])
recallClass2 = recall(myMatrix[2], myMatrix[3])
accuracyContNode = accuracy(myMatrix[0], myMatrix[1], myMatrix[2], myMatrix[3])
print "precision: "
print precisionClass1, precisionClass2
print "recall: "
print recallClass1, recallClass2
print "accuracy: "
print accuracyContNode
#print(myMatrix)
# Answer to Q7

precision: 
0.875748502994 0.612716763006
recall: 
0.897239263804 0.560846560847
accuracy: 
0.775267538644
